In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.init as init
import numpy as np
from tqdm import tqdm

from model.CustomDataSet import CustomDataSetRNN, CustomDataSet
from model.Tokenizer import SimpleTokenizerV1
from model.TTT import *
from environnement.environnement1Str import Environnement1
from environnement.environnement3Str import Environnement3
from environnement.environnement6Str import Environnement6
from environnement.small_loop import small_loop
from outil import *

In [ ]:
# init data
env_test = small_loop(x= 1, y=1, theta=0, world= np.array([
                [1, 1, 1, 1, 1],
                [1, 0, 0, 0, 1],
                # [1, 0, 1, 0, 1],
                # [1, 0, 0, 0, 1],
                [1, 1, 1, 1, 1],
            ]))
actions, outcomes = [], []

for i in range(50):
    action = str(np.random.choice(env_test.get_actions()))
    outcome = env_test.outcome(action)
    actions.append(action)
    outcomes.append(outcome)

print(actions)
print(outcomes)

tokenizer = SimpleTokenizerV1(vocab=
        create_dico_numerate_word(env_test.get_outcomes() + env_test.get_actions()))

data_set = CustomDataSet(actions=actions, outcomes=outcomes,
                    context_lenght=21, dim_out=, tokenizer=tokenizer)

data_loader_train = DataLoader(data_set, batch_size=32, shuffle=True, num_workers=4, drop_last=True)

# text, target = next(iter(data_loader_train))
# print(text)
# print(target)


for i in range(1000):
    action = str(np.random.choice(env_test.get_actions()))
    outcome = env_test.outcome(action)
    actions.append(action)
    outcomes.append(outcome)

data_set_test = CustomDataSet(actions=actions, outcomes=outcomes,
                    context_lenght=15, dim_out=2, tokenizer=tokenizer)

data_loader_test = DataLoader(data_set, batch_size=128, shuffle=True, num_workers=4, drop_last=True)

['turn_right', 'turn_right', 'feel_front', 'turn_left', 'turn_right', 'forward', 'turn_right', 'turn_left', 'turn_right', 'turn_right', 'turn_right', 'turn_left', 'forward', 'turn_left', 'feel_front', 'turn_left', 'turn_right', 'forward', 'turn_right', 'turn_left', 'turn_left', 'turn_right', 'forward', 'feel_front', 'forward', 'forward', 'turn_left', 'forward', 'feel_front', 'turn_right', 'turn_left', 'forward', 'feel_front', 'forward', 'feel_front', 'feel_front', 'feel_front', 'turn_left', 'forward', 'turn_right', 'feel_front', 'turn_left', 'turn_left', 'turn_left', 'turn_right', 'feel_front', 'feel_front', 'feel_front', 'feel_front', 'turn_left']
['empty', 'empty', 'wall', 'empty', 'empty', 'wall', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'wall', 'empty', 'wall', 'empty', 'empty', 'wall', 'empty', 'empty', 'empty', 'empty', 'wall', 'wall', 'wall', 'wall', 'empty', 'wall', 'wall', 'empty', 'empty', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'empty', 'empty', 'empty',

In [ ]:
# Exemple de configuration
config = TTTConfig(
    vocab_size=6,  # Exemple de taille de vocabulaire
    hidden_size=32,
    num_hidden_layers=6,
    num_attention_heads=8,
    max_position_embeddings=512,
    pretraining_tp=0.15,
)

model = TTTModel(config)

In [4]:
optimizer = optim.AdamW(model.parameters(), lr=0.01)

# Define the loss function as CrossEntropyLoss for classification
loss_fn = nn.CrossEntropyLoss()

In [9]:
model.train()

for epoch in range(10):
    for batch in data_loader_train:
        inputs, labels = batch
        labels = labels.to(torch.int64)
        print("inputs")
        print(inputs)
        print("labels")
        print(labels)
        outputs = model(inputs)
        print("outputs")
        print(outputs.hidden_states)
        print("outputs last_hidden_state")
        print(outputs.last_hidden_state)
        print(outputs.last_hidden_state.shape)
        print("outputs cache_params")
        print(outputs.cache_params)
        
        predi = torch.argmax(outputs.last_hidden_state, dim=1).to(torch.int64)
        print("predi")
        print(predi)
        # Pass the raw logits to the loss function
        loss = loss_fn(outputs.last_hidden_state, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

inputs
tensor([[2, 1, 2, 0, 2, 0, 3, 1, 3, 1, 3, 1, 2, 0, 4, 1, 4, 1, 2, 0, 4],
        [3, 1, 2, 1, 4, 1, 5, 0, 2, 0, 4, 1, 3, 1, 4, 1, 5, 1, 3, 1, 3],
        [5, 1, 5, 1, 5, 1, 2, 1, 3, 1, 3, 1, 4, 1, 2, 0, 5, 0, 4, 1, 5],
        [4, 1, 3, 1, 5, 0, 5, 0, 5, 0, 3, 1, 2, 0, 2, 0, 4, 1, 3, 1, 5],
        [3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2, 0, 3, 1, 5, 1, 4, 1, 5, 0, 5],
        [4, 1, 4, 1, 5, 0, 2, 0, 4, 1, 5, 1, 5, 1, 5, 1, 2, 1, 3, 1, 4],
        [2, 0, 3, 1, 3, 1, 4, 1, 5, 1, 2, 1, 3, 1, 3, 1, 3, 1, 5, 0, 5],
        [3, 1, 3, 1, 3, 1, 4, 1, 4, 1, 3, 1, 4, 1, 5, 1, 2, 1, 5, 1, 4],
        [5, 0, 3, 1, 3, 1, 3, 1, 2, 1, 5, 1, 4, 1, 2, 0, 4, 1, 4, 1, 3],
        [5, 0, 2, 0, 5, 0, 5, 0, 4, 1, 4, 1, 4, 1, 2, 0, 5, 0, 3, 1, 2],
        [3, 1, 4, 1, 3, 1, 2, 0, 5, 0, 4, 1, 5, 0, 3, 1, 3, 1, 3, 1, 4],
        [5, 0, 2, 0, 2, 0, 4, 1, 5, 1, 2, 1, 3, 1, 3, 1, 2, 1, 4, 1, 3],
        [3, 1, 5, 0, 2, 0, 3, 1, 2, 0, 3, 1, 2, 1, 4, 1, 2, 0, 2, 0, 3],
        [3, 1, 4, 1, 2, 0, 4, 1, 4, 1, 4, 1,

RuntimeError: Expected target size [32, 128], got [32]